In [7]:
import pandas as pd
import numpy as np
import requests
from time import sleep
import random 
from bs4 import BeautifulSoup
from collections import defaultdict
import re
import math

#### Use beatufull soup to get the number of articles per year
One of the parameters for NYTimes API request is 'page'. Page number has to be less than 200 => Need to break down by year

In [8]:
years = np.arange(1985,2019,1)

In [9]:
years

array([1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995,
       1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006,
       2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
       2018])

In [10]:
# define a function that inouts a year in the url and return the number of articles with a gives search query
def num_of_pages(year):
    url = f'https://www.nytimes.com/search/gun%20control/newest/{year}0101/{year}1231'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    response = requests.get(url, headers=headers)
    search_page = response.text
    soup = BeautifulSoup(search_page,"html5lib")
    search_result = soup.find_all(class_='SearchForm-searchStatus--2Z3Tw')
    search_result_str = str(search_result)
    num_of_articles = re.match(r'.*Showing\s(\d,*\d+)\sresults.*',search_result_str).group(1)
    num = int(num_of_articles.replace(',',''))
    return(num)

In [11]:
news_per_year = defaultdict()
for year in years:
    news_per_year[year]=num_of_pages(year)
    print(year, ': ', num_of_pages(year))

1985 :  279
1986 :  243
1987 :  217
1988 :  340
1989 :  357
1990 :  377
1991 :  412
1992 :  468
1993 :  539
1994 :  618
1995 :  431
1996 :  477
1997 :  369
1998 :  416
1999 :  828
2000 :  1009
2001 :  537
2002 :  513
2003 :  445
2004 :  498
2005 :  349
2006 :  463
2007 :  700
2008 :  478
2009 :  517
2010 :  462
2011 :  631
2012 :  836
2013 :  1418
2014 :  678
2015 :  817
2016 :  1226
2017 :  635
2018 :  761


#### Connect to NYTimes API to get article meta data

In [14]:
API_ROOT = 'https://api.nytimes.com/svc/search/v2/articlesearch.json'

API_SIGNUP_PAGE = 'http://developer.nytimes.com/docs/reference/keys'

In [15]:
# keys that stoppped working
# '05faa13dbd9f43f2baaa8cfb77fdbdf6'
# 'e4a266caf4924515a7fb9b093584f54b'
# 'a5bea43a4bc94a9ca93987ac86239055'
# '26a1d65efbed44a788dd16e4c41c6b11'

In [1]:
#keys = ['574c1decf6e34ee884d48718ac9420ee']
keys= ['fbdb0bf2114c49a4af9d5a810f5ad0d5']

from 01/01/1985 to date there are 19,294 articles that came up for 'gun control' search term.
it means i have to iterate through 1930 pages to collect the urls

In [2]:
# define a function that calls NYTimes API and returns a json object with sesarch page results 
# Note: returns meta data about 10 articles (1 search page)
def create_request(start_date, end_date, page_num):
    s = len(keys)-1
    API_KEY = keys[random.randint(0,s)]
    resp = requests.get(API_ROOT, params={
        'api-key': API_KEY,
        'q': "gun control",
        'begin_date': start_date,
        'end_date': end_date,
        'sort': "newest",
        'page': page_num})
    return(resp.json())

In [3]:
test = create_request('20130101', '20131231', 20)

NameError: name 'random' is not defined

In [4]:
test

NameError: name 'test' is not defined

In [24]:
# define a function that takes json output and parses data from it
# this will return article URLs for further scraping of articles' text
def parse_search_res(json_file):
    news = []
    for i in json_file['response']['docs']:   
        dic = {}
        dic['id'] = i['_id']
        dic['headline'] = i['headline']['main'].encode("utf8")
        dic['date'] = i['pub_date'][0:10]
        dic['score'] = i['score']
        if i['snippet'] is not None:
            dic['snippet'] = i['snippet'].encode("utf8")
        if 'source' in i:
            dic['source'] = i['source']
        if 'type_of_material' in i:
            dic['type'] = i['type_of_material']
        dic['url'] = i['web_url']
        dic['word_count'] = i['word_count']
        news.append(dic)
    return(news)

In [18]:
# define a function that takes the number of article during publication year 
# and splits them into chunks of 500 artciles. This is done becasue API is limiting number of calls and breaks 
# def find_slices(v):
#     if v<=500:
#         slices = [math.ceil(v/10)]
#         return([math.ceil(v/10)])
#     else:
#         slices = []
#         s = math.ceil(v/10)//50 + 1
#         for i in range(1,s):
#             slices.append(i*50)
#         last = slices[-1] + math.ceil(v/10)%50
#         slices.append(last)
#     return(slices)  

In [25]:
for k, v in news_per_year.items():
    data = []
    start_date = f'{k}0101'
    end_date = f'{k}1231'
    pages = math.ceil(v/10)+1
    for page in range(pages):
        data.extend(parse_search_res(create_request(start_date,end_date,page)))
        sleep(2)
    df = pd.DataFrame.from_dict(data)
    df.to_csv(f'nytimes_meta_{k}.csv')
    print('Collected:', k)

Collected: 1985


KeyboardInterrupt: 